In [82]:
import cv2
import numpy as np
import math
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# points in face:
# nose[0]
# l_eye_outer[3]
# r_eye_outer[6]
# mouth_left[9]
# mouth_right[10]

In [95]:
cap.release()
cv2.destroyAllWindows()

In [58]:
cap = cv2.VideoCapture(1)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    height, width, _ = image.shape
    size = image.shape
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    face_point_x = []
    face_point_y = []
    for i in range(11):
        face_point_x.append(results.pose_landmarks.landmark[i].x * width)
        face_point_y.append(results.pose_landmarks.landmark[i].y * height)
    face_point = list(zip(face_point_x, face_point_y))
    
    image_points = np.array([
        face_point[0],     # Nose tip

        face_point[3],     # Left eye left corner

        face_point[6],     # Right eye right corne

        face_point[9],     # Left Mouth corner

        face_point[10]      # Right mouth corner
    ], dtype="double")
    
    # 3D model points.

    model_points = np.array([
    
        (0.0, 0.0, 0.0),             # Nose tip
    
        (-225.0, 170.0, -135.0),     # Left eye left corner
    
        (225.0, 170.0, -135.0),      # Right eye right corne
    
        (-150.0, -150.0, -125.0),    # Left Mouth corner
    
        (150.0, -150.0, -125.0)      # Right mouth corner
    ])
    
    # Camera internals



    focal_length = size[1]

    center = (size[1]/2, size[0]/2)

    camera_matrix = np.array(

                             [[focal_length, 0, center[0]],

                             [0, focal_length, center[1]],

                             [0, 0, 1]], dtype = "double"

                             )

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP)
    (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)



    for p in image_points:
        cv2.circle(image, (int(p[0]), int(p[1])), 3, (0,0,255), -1)

    p1 = ( int(image_points[0][0]), int(image_points[0][1]))
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

    cv2.line(image, p1, p2, (255,0,0), 2)
    
#     for point in face_point:
#         cv2.circle(image, (int(point[0]), int(point[1])), 3, (0,0,255), -1)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#     mp_drawing.draw_landmarks(
#         image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow('MediaPipe Pose', image)
    if cv2.waitKey(5) & 0xFF == ord("q"):
      break
cap.release()
cv2.destroyAllWindows()

In [78]:
def draw_annotation_box(img, rotation_vector, translation_vector, camera_matrix, color=(255, 255, 0), line_width=2):
    """Draw a 3D box as annotation of pose"""
    point_3d = []
    dist_coeffs = np.zeros((4,1))
    rear_size = 1
    rear_depth = 0
    point_3d.append((-rear_size, -rear_size, rear_depth))
    point_3d.append((-rear_size, rear_size, rear_depth))
    point_3d.append((rear_size, rear_size, rear_depth))
    point_3d.append((rear_size, -rear_size, rear_depth))
    point_3d.append((-rear_size, -rear_size, rear_depth))

    front_size = img.shape[1]
    front_depth = front_size*2
    point_3d.append((-front_size, -front_size, front_depth))
    point_3d.append((-front_size, front_size, front_depth))
    point_3d.append((front_size, front_size, front_depth))
    point_3d.append((front_size, -front_size, front_depth))
    point_3d.append((-front_size, -front_size, front_depth))
    point_3d = np.array(point_3d, dtype=np.float32).reshape(-1, 3)

    # Map to 2d img points
    (point_2d, _) = cv2.projectPoints(point_3d,
                                      rotation_vector,
                                      translation_vector,
                                      camera_matrix,
                                      dist_coeffs)
    point_2d = np.int32(point_2d.reshape(-1, 2))
    

    # # Draw all the lines
    # cv2.polylines(img, [point_2d], True, color, line_width, cv2.LINE_AA)
    k = (point_2d[5] + point_2d[8])//2
    # cv2.line(img, tuple(point_2d[1]), tuple(
    #     point_2d[6]), color, line_width, cv2.LINE_AA)
    # cv2.line(img, tuple(point_2d[2]), tuple(
    #     point_2d[7]), color, line_width, cv2.LINE_AA)
    # cv2.line(img, tuple(point_2d[3]), tuple(
    #     point_2d[8]), color, line_width, cv2.LINE_AA)
    
    return(point_2d[2], k)

In [ ]:
# landmark in holistic:
# nose_tip = [1]
# chin = [152]
# l_eye_corner = [130]
# r_eye_corner = [359]
# l_mouth = [61]
# r_mouth = [291]
index_list = [1, 152, 130, 359, 61, 291]
font = cv2.FONT_HERSHEY_SIMPLEX 
mp_holistic = mp.solutions.holistic

#cap = cv2.VideoCapture(1)
cap = cv2.VideoCapture("../media/video/face_me.mp4")

with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        success, image = cap.read()
        height, width, _ = image.shape
        size = image.shape
        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        try:
            results = holistic.process(image)

            face_point_x = []
            face_point_y = []
            for i in index_list:
                face_point_x.append(results.face_landmarks.landmark[i].x * width)
                face_point_y.append(results.face_landmarks.landmark[i].y * height)
            face_point = list(zip(face_point_x, face_point_y))

            image_points = np.array([
                face_point[0],     # Nose tip

                face_point[1],     # Chin

                face_point[2],     # Left eye left corner

                face_point[3],     # Right eye right corne

                face_point[4],     # Left Mouth corner

                face_point[5]      # Right mouth corner
            ], dtype="double")

            # 3D model points.

            model_points = np.array([

                (0.0, 0.0, 0.0),             # Nose tip

                (0.0, -330.0, -65.0),        # Chin

                (-225.0, 170.0, -135.0),     # Left eye left corner

                (225.0, 170.0, -135.0),      # Right eye right corne

                (-150.0, -150.0, -125.0),    # Left Mouth corner

                (150.0, -150.0, -125.0)      # Right mouth corner
            ])

            # Camera internals



            focal_length = size[1]

            center = (size[1]/2, size[0]/2)

            camera_matrix = np.array(

                                     [[focal_length, 0, center[0]],

                                     [0, focal_length, center[1]],

                                     [0, 0, 1]], dtype = "double"

                                     )

            dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP)
            (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)



            for p in image_points:
                cv2.circle(image, (int(p[0]), int(p[1])), 3, (0,0,255), -1)

            p1 = ( int(image_points[0][0]), int(image_points[0][1]))
            p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
            x1, x2 = draw_annotation_box(image, rotation_vector, translation_vector, camera_matrix)

            cv2.line(image, p1, p2, (255,0,0), 2)
            cv2.line(image, (0, int(face_point[0][1])), (width, int(face_point[0][1])), (255, 0, 0), 2)
            if p2[1] > p1[1]:
                cv2.putText(image, "LOOKING DOWN", (20, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))
            #print(face_point[0][1])
            try:
                m = (p2[1] - p1[1])/(p2[0] - p1[0])
                ang1 = int(math.degrees(math.atan(m)))
            except:
                ang1 = 90

            try:
                m = (x2[1] - x1[1])/(x2[0] - x1[0])
                ang2 = int(math.degrees(math.atan(-1/m)))
            except:
                ang2 = 90

                    # print('div by zero error')
            #cv2.putText(image, str(ang1), tuple(p1), font, 2, (128, 255, 255), 3)
            cv2.putText(image, str(ang2), tuple(x1), font, 2, (255, 255, 128), 3)

        #     for point in face_point:
        #         cv2.circle(image, (int(point[0]), int(point[1])), 3, (0,0,255), -1)

            # Draw the pose annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        #     mp_drawing.draw_landmarks(
        #         image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        except:
            cv2.putText(image, "NO DETECTION", (20, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))
            pass
        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == ord("q"):
          break
cap.release()
cv2.destroyAllWindows()